# Multiclass Clustering

In [ ]:
import xgboost
import shap
print(xgboost.__version__)
print(shap.__version__)


In [ ]:
# % load xgboost_multiclass.py

In [ ]:
prs_pheno_clustered = pd.read_csv('', index_col = 'Unnamed: 0')
prot = pd.read_csv('', index_col = 'PMBB_ID')

demo_cols = ['Enrollment_age', 'gender_source_value', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5']
fin = prs_pheno_clustered[demo_cols + list(prot.columns) + ['SCORE1_AVG'] + ["Assigned_Cluster"]].dropna()

X = fin[demo_cols + list(prot.columns) + ['SCORE1_AVG']]
X = X.loc[:, ~X.columns.duplicated()]
y = fin["Assigned_Cluster"]

results = run_xgboost_multiclass(X, y, n_iter=10, output_prefix='', n_trials = 20)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

df = fin.copy()
df['PRS'] = df['SCORE1_AVG']  # adjust to your PRS column name
demo_cols = ['Enrollment_age', 'gender_source_value', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5']
X = df[demo_cols + list(prot.columns) + ['SCORE1_AVG'] + ["Assigned_Cluster"]].dropna()


# --- 1. Get PRS tertiles ---
df['PRS_tertile'] = pd.qcut(df['PRS'], q=3, labels=["Low", "Medium", "High"])

probs = pd.read_csv("", index_col = 'sample_id')
# Merge by sample index
merged = probs.join(df)

# Melt for plotting: long format
melted = merged.melt(id_vars=['PRS_tertile', 'iteration'],
                     value_vars=['prob_class_0', 'prob_class_1', 'prob_class_2', 'prob_class_3'],
                     var_name='Class', value_name='Predicted_Prob')

In [ ]:
pd.crosstab(merged['true'], merged['PRS_tertile'], normalize='columns')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# freqs from crosstab
data = {
    'Low':    [0.288043, 0.277174, 0.177536, 0.257246],
    'Medium': [0.362173, 0.285714, 0.144869, 0.207243],
    'High':   [0.397590, 0.283993, 0.086059, 0.232358]
}
classes = ['Class 0', 'Class 1', 'Class 2', 'Class 3']
df = pd.DataFrame(data, index=classes)

# Set color palette for classes from clustering
cluster_palette = {
    'Class 0': sns.color_palette("Set2")[0],
    'Class 1': sns.color_palette("Set2")[1],
    'Class 2': sns.color_palette("Set2")[2],
    'Class 3': sns.color_palette("Set2")[3]
}

# Plot
fig, axes = plt.subplots(3, 1, figsize=(10, 20), sharey=False)

for ax, tertile in zip(axes, df.columns):
    ax.bar(df.index, df[tertile], color=[cluster_palette[cls] for cls in df.index])
    ax.set_title(f"PRS Tertile: {tertile}", fontsize = 20)
    ax.set_ylabel("Proportion", fontsize = 18)
    ax.set_ylim(0, 0.5)
    ax.tick_params(axis='both', rotation=0, labelsize=16)

fig.suptitle("Class Proportions Across PRS Tertiles", fontsize=30)
plt.show()


In [ ]:
# %load multinomial_regression_multiclass.py

In [ ]:
prs_pheno_clustered = pd.read_csv('', index_col = 'Unnamed: 0')
prot = pd.read_csv('', index_col = 'PMBB_ID')

demo_cols = ['Enrollment_age', 'gender_source_value', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5'] + list(prot.columns)
fin = prs_pheno_clustered[demo_cols + list(prot.columns) + ['SCORE1_AVG'] + ["Assigned_Cluster"]].dropna()

X = fin[demo_cols + list(prot.columns) + ['SCORE1_AVG']]
X = X.loc[:, ~X.columns.duplicated()]
y = fin["Assigned_Cluster"]

results = run_xgboost_multiclass(X, y, n_iter=10, output_prefix='', n_trials = 20)